# Библиотеки Python для DS (семинары)

Урок 9. Использование на практике методов снижения размерности

Использование алгоритмов понижения размерности для улучшения классификации новостей (https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html)

Цель задания: Исследовать влияние различных методов понижения размерности на качество классификации текстовых данных.

Датасет: Набор данных новостных статей
(датасет '20 Newsgroups' доступный в sklearn.datasets).

Задачи:

1. Загрузите датасет '20 Newsgroups' из sklearn.

2. Проведите предобработку данных (очистка текста, удаление стоп-слов, векторизация с использованием TF-IDF).

3. Примените к полученным векторам TF-IDF следующие методы понижения размерности:
— PCA (Principal Component Analysis)
— t-SNE (t-distributed Stochastic Neighbor Embedding)
— UMAP (Uniform Manifold Approximation and Projection).

4. После понижения размерности данных используйте любой метод машинного обучения для классификации новостей по темам.

5. Сравните качество классификации для каждого метода понижения размерности. Используйте метрики точности и F1-меру.

6. Визуализируйте двумерное представление данных для каждого метода понижения размерности, чтобы оценить, как алгоритмы справляются с сепарацией классов.

7. Напишите отчёт, в котором обсудите, какой метод понижения размерности оказал наиболее значительное влияние на качество классификации и почему.


## Выполнение домашней работы

##### Используем предоставленый к заданию файл fetch_california_housing.xlsx

In [ ]:
# Библиотека для загрузки датасета
from sklearn.datasets import fetch_20newsgroups
import joblib
# Библиотеки для работы с датасетом и графиками
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.tree import DecisionTreeRegressor

##### 1. Загрузите датасет '20 Newsgroups' из sklearn.

In [ ]:
# Загрузка датасета
newsgroups = fetch_20newsgroups(subset='all')

# Просмотр информации о датасете
print(f"Количество документов: {len(newsgroups.data)}")
print(f"Количество классов: {len(newsgroups.target_names)}")

Количество документов: 18846
Количество классов: 20


In [7]:
# Указываем путь к папке
output_dir = '20_newsgroups_data'
os.makedirs(output_dir, exist_ok=True)

# Сохраняем документы в текстовые файлы
for i, (doc, label) in enumerate(zip(newsgroups.data, newsgroups.target)):
    # Создаем имя файла на основе индекса документа
    file_name = os.path.join(output_dir, f'document_{i}.txt')
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(doc)

# Сохраняем метки классов в отдельный файл
labels_file_path = os.path.join(output_dir, 'labels.txt')
with open(labels_file_path, 'w', encoding='utf-8') as f:
    for label in newsgroups.target:
        f.write(f"{label}\n")

print(f"Датасет сохранен в папке: {output_dir}")

Датасет сохранен в папке: 20_newsgroups_data


In [ ]:
# Загрузка датасета
newsgroups_data = fetch_20newsgroups(subset='all')

# Сохранение данных в файл
joblib.dump(newsgroups_data, '20_newsgroups_data.pkl')

['20_newsgroups_data.pkl']

In [ ]:
# Загрузка данных из файла
# newsgroups_data = joblib.load('20_newsgroups_data.pkl')
newsgroups_data = fetch_20newsgroups(subset='all')

# Создание DataFrame
data = {
    'text': newsgroups_data.data,
    'target': newsgroups_data.target}
df = pd.DataFrame(data)

# Добавление категорий
df['category'] = df['target'].apply(lambda x: newsgroups_data.target_names[x])

# Вывод DataFrame
df

,text,target,category
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10,rec.sport.hockey
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3,comp.sys.ibm.pc.hardware
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17,talk.politics.mideast
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3,comp.sys.ibm.pc.hardware
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4,comp.sys.mac.hardware
...,...,...,...
18841,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
18842,From: rdell@cbnewsf.cb.att.com (richard.b.dell...,12,sci.electronics
18843,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
18844,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


##### 2. Проведите предобработку данных (очистка текста, удаление стоп-слов, векторизация с использованием TF-IDF).

##### 3. Примените к полученным векторам TF-IDF следующие методы понижения размерности:

— PCA (Principal Component Analysis)

— t-SNE (t-distributed Stochastic Neighbor Embedding)

— UMAP (Uniform Manifold Approximation and Projection).

##### 4. После понижения размерности данных используйте любой метод машинного обучения для классификации новостей по темам.

##### 5. Сравните качество классификации для каждого метода понижения размерности. Используйте метрики точности и F1-меру.

##### 6. Визуализируйте двумерное представление данных для каждого метода понижения размерности, чтобы оценить, как алгоритмы справляются с сепарацией классов.

##### 7. Напишите отчёт, в котором обсудите, какой метод понижения размерности оказал наиболее значительное влияние на качество классификации и почему.